In [13]:
import pandas as pd
import sklearn as skl

In [20]:
data = pd.read_csv("result.csv")
df = pd.DataFrame(data)
print(df.head())
type(df["asset_id"][0])

   Unnamed: 0               OBJID  SPIRAL  ELLIPTICAL  UNCERTAIN  asset_id
0           0  587731186203885750       0           0          1    278415
1           1  587731187277627676       0           0          1    280228
2           2  588015507658768464       0           0          1    287951
3           3  587731187277693069       0           1          0    280231
4           4  587731187277693072       0           0          1    280232


numpy.int64

In [21]:
from sklearn.model_selection import train_test_split
df_sampled = df.sample(n=10000, random_state=42)
x_train,x_test,y_train,y_test = train_test_split(df_sampled["asset_id"],df_sampled[["SPIRAL","ELLIPTICAL","UNCERTAIN"]],train_size=0.8,random_state=55)

print(x_train.shape)
print(x_test.shape)
print(y_train)
print(y_test.shape)


(8000,)
(2000,)
        SPIRAL  ELLIPTICAL  UNCERTAIN
192954       0           0          1
58065        0           0          1
247063       0           0          1
74741        0           0          1
29849        1           0          0
...        ...         ...        ...
192670       0           1          0
134112       0           0          1
218236       0           0          1
181974       0           0          1
171421       1           0          0

[8000 rows x 3 columns]
(2000, 3)


In [16]:
import os 
import shutil

source_dir = "E:/Coding Folder/Projects/Galaxy_img/images_gz2/images"
train_dir = "train_dataset"
test_dir = "test_dataset"
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for img_name in x_train:
    try:
        src_path = os.path.join(source_dir, f"{str(img_name)}.jpg")
        dest_path = os.path.join(train_dir, f"{str(img_name)}.jpg")
        
        if os.path.exists(src_path):
            shutil.copy2(src_path, dest_path)
            print(f"Copied {img_name}")
        else:
            print(f"Source file not found: {src_path}")
            
    except Exception as e:
        print(f"Error copying {img_name}: {str(e)}")

for img_name in x_test:
    try:
        src_path = os.path.join(source_dir, f"{str(img_name)}.jpg")
        dest_path = os.path.join(test_dir, f"{str(img_name)}.jpg")
        
        if os.path.exists(src_path):
            shutil.copy2(src_path, dest_path)
            print(f"Copied {img_name}")
        else:
            print(f"Source file not found: {src_path}")
            
    except Exception as e:
        print(f"Error copying {img_name}: {str(e)}")

Copied 17695
Copied 194747
Copied 291612
Copied 162303
Copied 153136
Copied 167226
Copied 197183
Copied 35840
Copied 275841
Copied 241069
Copied 209989
Copied 88703
Copied 92693
Copied 59936
Copied 70406
Copied 41519
Copied 29751
Copied 77323
Copied 61481
Copied 79710
Copied 223740
Copied 98847
Copied 91700
Copied 175851
Copied 18206
Copied 126773
Copied 138786
Source file not found: E:/Coding Folder/Projects/Galaxy_img/images_gz2/images\283493.jpg
Copied 277055
Copied 117074
Copied 140544
Copied 190786
Copied 142096
Copied 130142
Copied 180063
Copied 204866
Copied 219070
Copied 251492
Copied 140795
Copied 154998
Copied 111058
Copied 30849
Copied 134027
Copied 291386
Copied 245117
Copied 92661
Copied 8706
Copied 155441
Copied 270361
Copied 3067
Copied 22620
Copied 98998
Copied 31248
Source file not found: E:/Coding Folder/Projects/Galaxy_img/images_gz2/images\294925.jpg
Copied 268739
Copied 4004
Copied 83376
Copied 217023
Copied 217396
Copied 60768
Copied 17866
Copied 208887
Copied 179

In [17]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

SIZE = (224, 224)
INPUT_ROOT = Path("train_dataset")   
OUTPUT_ROOT = Path("resized_dataset")

def process_folder(inp_folder: Path, out_folder: Path):
    out_folder.mkdir(parents=True, exist_ok=True)
    for img_path in tqdm(list(inp_folder.iterdir()), desc=inp_folder.name):
        if img_path.suffix.lower() not in {".jpg",".jpeg",".png"}:
            continue
        with Image.open(img_path) as img:
            img = img.resize(SIZE, Image.BILINEAR).convert("L")
            img.save(out_folder / img_path.name)

for split in ("train_dataset", "test_dataset"):
    process_folder(Path(split), OUTPUT_ROOT / split)

print("Done! Look in:", OUTPUT_ROOT)


test_dataset: 100%|██████████| 1937/1937 [00:18<00:00, 106.06it/s]

Done! Look in: resized_dataset


In [18]:
import os
import cv2
import numpy as np

def load_images(asset_ids, folder_path):
    images = []
    for asset_id in asset_ids:
        img_path = os.path.join(folder_path, f"{asset_id}.jpg")
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"⚠️ Warning: Could not read image {img_path}, skipping.")
            continue

        img = img / 255.0
        img = np.expand_dims(img, axis=-1)
        images.append(img)
    return np.array(images)

def clean_asset_ids(asset_ids, labels, folder_path):
    cleaned_ids = []
    cleaned_labels = []

    for asset_id, label in zip(asset_ids, labels):
        img_path = os.path.join(folder_path, f"{asset_id}.jpg")
        if os.path.exists(img_path):
            cleaned_ids.append(asset_id)
            cleaned_labels.append(label)

    return np.array(cleaned_ids), np.array(cleaned_labels)


In [19]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout
)

model = Sequential([
    Input(shape=(224,224,1)),

    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),

    Dense(3, activation='softmax'),
])

model.summary()

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = Adam(learning_rate=0.01)
)

# if isinstance(y_train[0], str) and y_train[0] in ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']:
#     y_train = y_train[1:]
#     x_train = x_train[1:]

# if isinstance(y_test[0], str) and y_test[0] in ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']:
#     y_test = y_test[1:]
#     x_test = x_test[1:]

x_train, y_train = clean_asset_ids(x_train, y_train,"E:/Coding Folder/Projects/ML-Project/resized_dataset/train_dataset")
x_test, y_test = clean_asset_ids(x_test, y_test,"E:/Coding Folder/Projects/ML-Project/resized_dataset/test_dataset")


x_train_imgs = load_images(x_train, "E:/Coding Folder/Projects/ML-Project/resized_dataset/train_dataset")
x_test_imgs = load_images(x_test, "E:/Coding Folder/Projects/ML-Project/resized_dataset/test_dataset")

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# print(y_train)

model.fit(
    x_train_imgs, y_train,
    validation_data=(x_test_imgs, y_test),
    epochs=10,
    batch_size=32
)


predictions = model.predict(x_test)
pred_class = np.argmax(predictions, axis=1)  


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,783,811 (98.36 MB)

 Trainable params: 25,783,811 (98.36 MB)

 Non-trainable params: 0 (0.00 B)

ValueError: could not convert string to float: np.str_('SPIRAL')

In [ ]:
print(y_train)

['SPIRAL' 'ELLIPTICAL' 'UNCERTAIN']
